In [ ]:
# Global State Bandit Simulation - Fastest Python Version with GPU Support

# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# GPU available?
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Cell 2: Parameters
K = 3
N = 1000

theta = torch.tensor([
    [0.1, 0.9],
    [0.5, 0.7],
    [0.9, 0.1]
], device=device)

pi_indiv = torch.tensor([
    [[0.9, 0.1], [0.1, 0.9]],
    [[0.85, 0.15], [0.2, 0.8]],
    [[0.95, 0.05], [0.3, 0.7]]
], device=device)

# Cell 3: Simulate Ground Truth Data
def simulate_bandit_data(K, N, theta, pi_indiv):
    z = torch.zeros((K, N), dtype=torch.int64, device=device)
    y = torch.zeros((K, N), dtype=torch.int64, device=device)

    for i in range(K):
        z[i, 0] = torch.bernoulli(torch.tensor(0.5, device=device)).long()
        y[i, 0] = torch.bernoulli(theta[i, z[i, 0]])
        for t in range(1, N):
            prob = pi_indiv[i, z[i, t-1], 1]
            z[i, t] = torch.bernoulli(prob).long()
            y[i, t] = torch.bernoulli(theta[i, z[i, t]])
    return y, z

# Cell 4: Generate and Store Data
y_true, z_true = simulate_bandit_data(K, N, theta, pi_indiv)

# Cell 5: Baseline Thompson Sampling
def baseline_thompson(K, N, y_true, z_true, theta):
    rewards = torch.zeros(K, device=device)
    counts = torch.ones(K, device=device)
    cum_regret = torch.zeros(N, device=device)
    cum_reward = torch.zeros(N, device=device)
    selected = torch.zeros(N, dtype=torch.int64, device=device)

    for t in range(N):
        samples = torch.distributions.Beta(rewards + 1, counts - rewards + 1).sample()
        a_t = torch.argmax(samples).item()

        r = y_true[a_t, t].float()
        rewards[a_t] += r
        counts[a_t] += 1

        oracle = torch.argmax(torch.tensor([theta[k, z_true[k, t]] for k in range(K)], device=device))
        opt_reward = theta[oracle, z_true[oracle, t]]
        regret = opt_reward - theta[a_t, z_true[a_t, t]]

        cum_reward[t] = rewards.sum()
        cum_regret[t] = cum_regret[t - 1] + regret if t > 0 else regret
        selected[t] = a_t

    return cum_reward.cpu().numpy(), cum_regret.cpu().numpy(), selected.cpu().numpy()

# Cell 6: Run Baseline Thompson
c_reward, c_regret, chosen = baseline_thompson(K, N, y_true, z_true, theta)

# Cell 7: Plot Results
plt.figure(figsize=(12, 5))
plt.plot(c_reward, label='Cumulative Reward', color='green')
plt.title('Baseline Thompson Sampling - Reward')
plt.xlabel('Time')
plt.ylabel('Cumulative Reward')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(c_regret, label='Cumulative Regret', color='red')
plt.title('Baseline Thompson Sampling - Regret')
plt.xlabel('Time')
plt.ylabel('Cumulative Regret')
plt.legend()
plt.grid(True)
plt.show()

# Cell 8: Expected Regret (Optional Multiple Runs)
def run_expectation(K, N, theta, pi_indiv, runs=100):
    rewards_all = []
    regrets_all = []
    for _ in range(runs):
        y, z = simulate_bandit_data(K, N, theta, pi_indiv)
        r, rg, _ = baseline_thompson(K, N, y, z, theta)
        rewards_all.append(r)
        regrets_all.append(rg)
    return np.mean(rewards_all, axis=0), np.mean(regrets_all, axis=0)

# Cell 9: Run and Plot Expectation
exp_reward, exp_regret = run_expectation(K, N, theta, pi_indiv, runs=100)

plt.figure(figsize=(12, 5))
plt.plot(exp_reward, label='Expected Reward', color='green')
plt.title('Expected Reward over Time (100 Runs)')
plt.xlabel('Time')
plt.ylabel('Expected Cumulative Reward')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(exp_regret, label='Expected Regret', color='red')
plt.title('Expected Regret over Time (100 Runs)')
plt.xlabel('Time')
plt.ylabel('Expected Cumulative Regret')
plt.grid(True)
plt.show()